In [2]:
# ORDER Types
# buy
# sell
# stoploss
# cancel

# no short selling
# 1 hour real time data = 1 second in simulation
# find data source?

# test diff to real stocks?
# Real day has different stocks and data to the testing, so could potentially find it but they are anonymised...
# Scale the testing data by 3600 then match it to known hitorical data
# Data given in pandas dataframe
# all trades made at bid (for selling our stock) and ask (for buying) prices
# no fee to trade
# infinite stocks, we arent trading with other competitors. Everything is pre calculated.
# Account can go to zero, finite money. Certain judges have good will which they can give money to you if feeling kind.
# Can buy unlimited volume, so good strategy is to buy extremely large amount of safe stock 

# Import notes:
# the live data as per the API has accuracy to two DP, but the historic csv has much more. ---> careful when training


# IDEAS
# - during the event we can train models off the live data and update incrementally
# want everything running concurrently to improve speed

In [3]:
import hackathon_linc as lh
import pandas as pd
import time
import asyncio

from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

In [4]:
lh.init('265d0b0b-7e97-44a7-9576-47789e8712b2')



    Welcome to the LINC Hackathon! Your token is now saved in the Console. 
    That means you don't need to carry that out when using the other functions
    as long as you don't close your console. 
    
    This function is only to be used once to authenticate your token.
    
    Happy coding!

    


In [ ]:
start_time = time.time()
orders = lh.get_all_orders()
print(f"Orders: {orders} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
completed = lh.get_completed_orders()
print(f"Completed Orders: {completed} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
pending = lh.get_pending_orders()
print(f"Pending Orders: {pending} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
stoploss_orders = lh.get_stoploss_orders()
print(f"Stoploss Orders: {stoploss_orders} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
balance = lh.get_balance()
print(f"Balance: {balance} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
portfolio = lh.get_portfolio()
print(f"Portfolio: {portfolio} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
all_tickers = lh.get_all_tickers()
print(f"All Tickers: {all_tickers} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
price_data = pd.DataFrame(lh.get_current_price()['data'])
display(price_data)
print(f"Time taken: {time.time() - start_time:.2f} seconds")


Orders: [{'amount': 1, 'created_at': '2015-12-18T07:51:00', 'ended_at': '2015-12-18T07:51:00', 'id': 'fed18efa-7a3b-4bce-84bd-8ec4336d250a', 'order_status': 'completed', 'order_type': 'buy', 'price': 91.247, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062ef4f', 'symbol': 'STOCK1', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}, {'amount': 3, 'created_at': '2016-04-14T11:30:00', 'ended_at': '2016-04-14T11:30:00', 'id': '2d3a5c26-6086-4704-b48b-c56060ae1df8', 'order_status': 'completed', 'order_type': 'buy', 'price': 78.28244166666667, 'security_id': 'dc708dd9-f3a0-4a00-b871-43e58d4948e1', 'symbol': 'STOCK3', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}, {'amount': 20, 'created_at': '2018-01-01T15:57:00', 'ended_at': '2018-01-01T15:57:00', 'id': 'efe404f2-f377-447c-b83e-d70f47765eeb', 'order_status': 'completed', 'order_type': 'buy', 'price': 91.745, 'security_id': '595b7464-c391-4e71-a393-2321b8f57dd3', 'symbol': 'STOCK2', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}, {

,askMedian,askVolume,bidMedian,bidVolume,gmtTime,spreadMedian,symbol
0,198.25,1000.0,198.15,1000.0,2018-12-12 07:00:00,0.106,STOCK1
1,146.58,1000.0,146.52,1000.0,2018-12-12 07:00:00,0.064,STOCK10
2,59.19,1000.0,59.17,1000.0,2018-12-12 07:00:00,0.026,STOCK2
3,71.56,1000.0,71.53,1000.0,2018-12-12 07:00:00,0.036,STOCK3
4,59.26,1000.0,59.16,1000.0,2018-12-12 07:00:00,0.102,STOCK4
5,7.47,1000.0,7.45,1000.0,2018-12-12 07:00:00,0.014,STOCK5
6,100.05,1000.0,99.98,1000.0,2018-12-12 07:00:00,0.074,STOCK6
7,18.00,1000.0,17.98,1000.0,2018-12-12 07:00:00,0.021,STOCK7
8,90.29,1000.0,90.25,1000.0,2018-12-12 07:00:00,0.046,STOCK8
9,98.44,1000.0,98.41,1000.0,2018-12-12 07:00:00,0.036,STOCK9


Time taken: 30.41 seconds


In [6]:
#start_time = time.time()

def sell_all():
    port = lh.get_portfolio()
    for key, quantity in port.items():
        lh.sell(key, quantity)
sell_all()
portfolio = lh.get_portfolio()
print(portfolio)




{'STOCK1': 0, 'STOCK10': 0, 'STOCK2': 0, 'STOCK3': 0, 'STOCK4': 0, 'STOCK5': 0, 'STOCK6': 0, 'STOCK7': 0, 'STOCK8': 0, 'STOCK9': 0}


In [ ]:
def buy_stock():
    return lh.buy("STOCK1", 1, None, None)

def sell_stock():
    return lh.sell("STOCK1", 1, None, None)

max_workers = 1

for i in range(20):
    print(f"~~~~~~~~~~~~~~~~{i}~~~~~~~~~~~~~~~~")
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=max_workers*2) as executor:
        futures = [executor.submit(buy_stock) for _ in range(max_workers)] + [executor.submit(sell_stock) for _ in range(max_workers)]
        for future in as_completed(futures):
            print(future.result())
    elapsed_time = time.time() - start_time
    if elapsed_time < 1:
        time.sleep(1 - elapsed_time)
    
    

    
    

~~~~~~~~~~~~~~~~0~~~~~~~~~~~~~~~~
{'amount': 1, 'created_at': '2015-12-23T15:25:00', 'ended_at': '2015-12-23T15:25:00', 'id': '0cccafd4-e356-4b89-a545-f8cd247c62f6', 'order_status': 'completed', 'order_type': 'buy', 'price': 90.13358333333332, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062ef4f', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}
{'amount': 1, 'created_at': '2015-12-23T15:31:00', 'ended_at': '2015-12-23T15:32:00', 'id': 'c2808238-82b6-4aab-a1d7-ba2b104858de', 'order_status': 'completed', 'order_type': 'sell', 'price': 90.13358333333332, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062ef4f', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}
~~~~~~~~~~~~~~~~1~~~~~~~~~~~~~~~~
{'message': 'Something else went wrong'}
{'amount': 1, 'created_at': '2015-12-24T07:13:00', 'ended_at': '2015-12-24T07:14:00', 'id': 'cc288297-dc70-412a-80c1-4185f9eb6b2f', 'order_status': 'completed', 'order_type': 'buy', 'price': 90.17190322580645, 'security_id': '09335597-8a4e-4f8e-8850-04c5e062

In [4]:
lh.buy("STOCK3", 3, None, None)

{'amount': 3,
 'created_at': '2016-04-14T11:30:00',
 'ended_at': '2016-04-14T11:30:00',
 'id': '2d3a5c26-6086-4704-b48b-c56060ae1df8',
 'order_status': 'completed',
 'order_type': 'buy',
 'price': 78.28244166666667,
 'security_id': 'dc708dd9-f3a0-4a00-b871-43e58d4948e1',
 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}

In [6]:
lh.sell("STOCK3", 3, None, None)

{'amount': 3,
 'created_at': '2016-04-18T09:03:00',
 'ended_at': '2016-04-18T09:03:00',
 'id': '9a0d9535-8752-44d9-b725-4c7baa3dff0a',
 'order_status': 'completed',
 'order_type': 'sell',
 'price': 79.2978,
 'security_id': 'dc708dd9-f3a0-4a00-b871-43e58d4948e1',
 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}

In [13]:
def my_buy(self, ticker, amount, price=None, days_to_cancel=1):
    params = {'type': 'buy',
            'ticker': ticker,
            'amount': amount,
            'days_to_cancel': days_to_cancel}
    body = {'api_key': "265d0b0b-7e97-44a7-9576-47789e8712b2"}

    if price is not None:
        params['price'] = price

    url_s = 'https://hackathonlincapp.azurewebsites.net/api' + '/order/'

    response = requests.post(url_s, params=params, json=body)
    response_json = response.json()
    print(response_json)
    if 'order_status' in response_json and response_json['order_status'] == 'completed':
        self.balance -= amount * price
        self.portfolio[ticker] += amount
        print(f"Bought {amount} of {ticker} at {price}")
        return response_json['order_status']

In [14]:
my_buy(None, 'STOCK9', 2)

{'amount': 2, 'created_at': '2017-12-15T09:10:00', 'ended_at': '2017-12-15T09:11:00', 'id': '9d48924c-4c21-4df4-8ac3-c33216daacac', 'order_status': 'completed', 'order_type': 'buy', 'price': 116.39069166666668, 'security_id': 'c536e571-e683-414a-9b8c-c3057c9da443', 'team_id': 'c72a4076-96d3-4b7a-8374-36ec26c1c846'}


AttributeError: 'NoneType' object has no attribute 'balance'

In [3]:
portfolio = lh.get_portfolio()
portfolio

{'STOCK1': 60}

In [15]:
start_time = time.time()
all_tickers = lh.get_all_tickers()
print(f"All Tickers: {all_tickers} (Time taken: {time.time() - start_time:.2f} seconds)")

start_time = time.time()
price_data = pd.DataFrame(lh.get_current_price()['data'])
display(price_data)
print(f"Time taken: {time.time() - start_time:.2f} seconds")



All Tickers: ['STOCK1', 'STOCK10', 'STOCK2', 'STOCK3', 'STOCK4', 'STOCK5', 'STOCK6', 'STOCK7', 'STOCK8', 'STOCK9'] (Time taken: 0.50 seconds)


,askMedian,askVolume,bidMedian,bidVolume,gmtTime,spreadMedian,symbol
0,194.14,8.470579e+05,194.05,1.128485e+06,2018-05-29 07:00:00,0.093575,STOCK1
1,164.12,1.152185e+06,164.04,1.244875e+06,2018-05-29 07:00:00,0.079950,STOCK10
2,86.01,3.114680e+06,85.97,3.002864e+06,2018-05-29 07:00:00,0.041175,STOCK2
3,86.30,1.940043e+06,86.25,7.433889e+05,2018-05-29 07:00:00,0.047075,STOCK3
4,76.51,3.551710e+05,76.44,4.506653e+05,2018-05-29 07:00:00,0.069192,STOCK4
5,10.07,3.938626e+05,10.05,3.149557e+05,2018-05-29 07:00:00,0.020242,STOCK5
6,107.67,4.825475e+05,107.59,5.630339e+05,2018-05-29 07:00:00,0.082908,STOCK6
7,23.50,2.727034e+06,23.48,3.606098e+06,2018-05-29 07:00:00,0.024183,STOCK7
8,95.51,3.621687e+06,95.46,4.090485e+06,2018-05-29 07:00:00,0.053058,STOCK8
9,113.02,1.890782e+06,112.98,1.810621e+06,2018-05-29 07:00:00,0.046550,STOCK9


Time taken: 15.73 seconds


In [17]:
historical_data = pd.DataFrame(lh.get_historical_data(100))
print(historical_data)

       askMedian     askVolume   bidMedian     bidVolume              gmtTime  \
0     182.556150  7.071134e+05  182.465817  5.676579e+05  2018-07-09 07:00:00   
1      75.249900  3.356290e+05   75.165750  2.898579e+05  2018-07-09 07:00:00   
2      80.303617  9.235212e+05   80.260550  1.068094e+06  2018-07-09 07:00:00   
3      21.829883  2.835315e+06   21.807367  2.978337e+06  2018-07-09 07:00:00   
4     100.063967  2.396090e+06  100.015700  2.608727e+06  2018-07-09 07:00:00   
...          ...           ...         ...           ...                  ...   
6295  195.087700  8.651672e+05  195.008867  7.915657e+05  2018-10-15 15:00:00   
6296   61.879400  2.793630e+05   61.846800  2.330640e+05  2018-10-15 15:00:00   
6297   18.112000  2.437843e+05   18.100133  1.355135e+05  2018-10-15 15:00:00   
6298   69.143100  4.736622e+05   69.122533  6.187216e+05  2018-10-15 15:00:00   
6299  144.281800  6.737417e+05  144.234133  2.332668e+05  2018-10-15 15:00:00   

      spreadMedian   symbol

In [18]:
lh.get_historical_data(25)

KeyboardInterrupt: 

### Overall controller class

In [13]:
class DataPoller:
    def __init__(self, polling_intervals):
        self.polling_intervals = polling_intervals
        self.cache = {}
        self.running = False
        self.executor = ThreadPoolExecutor(max_workers=len(polling_intervals))

    def poll_data(self, func, func_name, interval):
        while self.running:
            start_time = time.time()
            result = func()
            self.cache[func_name] = result
            elapsed_time = time.time() - start_time
            print(f"{func_name}: {result} (Time taken: {elapsed_time:.2f} seconds)")
            time.sleep(interval)

    def start_polling(self):
        self.running = True
        for func_name, interval in self.polling_intervals.items():
            func = getattr(lh, func_name)
            self.executor.submit(self.poll_data, func, func_name, interval)

    def stop_polling(self):
        self.running = False
        self.executor.shutdown(wait=False)

    def get_cached_data(self, func_name):
        return self.cache.get(func_name)

def strategy_function(data_poller):
    while True:
        # Access cached data
        all_orders = data_poller.get_cached_data("get_all_orders")
        completed_orders = data_poller.get_cached_data("get_completed_orders")
        pending_orders = data_poller.get_cached_data("get_pending_orders")
        stoploss_orders = data_poller.get_cached_data("get_stoploss_orders")
        balance = data_poller.get_cached_data("get_balance")
        portfolio = data_poller.get_cached_data("get_portfolio")
        all_tickers = data_poller.get_cached_data("get_all_tickers")
        current_price = data_poller.get_cached_data("get_current_price")

        # Implement your strategy logic here
        print(f"Strategy accessing data: {all_orders}, {completed_orders}, {pending_orders}, {stoploss_orders}, {balance}, {portfolio}, {all_tickers}, {current_price}")

        time.sleep(1)

# Example usage
if __name__ == "__main__":
    polling_intervals = {
        "get_all_orders": 5,
        "get_completed_orders": 10,
        "get_pending_orders": 15,
        "get_stoploss_orders": 5,
        "get_balance": 10,
        "get_portfolio": 15,
        "get_all_tickers": 2,
        "get_current_price": 1,
    }

    data_poller = DataPoller(polling_intervals)

    try:
        data_poller.start_polling()
        while True:
            time.sleep(1)
            print(1)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(2)
            print(2)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(3)
            print(3)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(4)
            print(4)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(5)
            print(5)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
            time.sleep(6)
            print(6)
            print("FROM main" + str(data_poller.get_cached_data("get_current_price")))
    except KeyboardInterrupt:
        data_poller.stop_polling()

get_completed_orders: [] (Time taken: 0.24 seconds)
get_pending_orders: [] (Time taken: 0.29 seconds)
get_portfolio: {} (Time taken: 0.29 seconds)
get_all_tickers: ['STOCK1', 'STOCK10', 'STOCK2', 'STOCK3', 'STOCK4', 'STOCK5', 'STOCK6', 'STOCK7', 'STOCK8', 'STOCK9'] (Time taken: 0.30 seconds)
get_stoploss_orders: [] (Time taken: 0.31 seconds)
get_current_price: {'data': [{'askMedian': 87.54, 'askVolume': 273.35, 'bidMedian': 87.51, 'bidVolume': 297.5833333333333, 'gmtTime': '2015-12-10 09:00:00', 'spreadMedian': 0.036725, 'symbol': 'STOCK1'}, {'askMedian': 135.8, 'askVolume': 121964.25022333332, 'bidMedian': 135.69, 'bidVolume': 70119.33303166667, 'gmtTime': '2015-12-10 09:00:00', 'spreadMedian': 0.110275, 'symbol': 'STOCK10'}, {'askMedian': 70.75, 'askVolume': 100278.11620666667, 'bidMedian': 70.73, 'bidVolume': 94534.33401833332, 'gmtTime': '2015-12-10 09:00:00', 'spreadMedian': 0.02165, 'symbol': 'STOCK2'}, {'askMedian': 97.39, 'askVolume': 44902.78333666667, 'bidMedian': 97.33, 'bid

get_pending_orders: [] (Time taken: 0.16 seconds)
get_portfolio: {} (Time taken: 0.12 seconds)
get_completed_orders: [] (Time taken: 0.20 seconds)
get_balance: 1000000.0 (Time taken: 0.14 seconds)
get_all_orders: [] (Time taken: 0.32 seconds)
get_stoploss_orders: [] (Time taken: 0.17 seconds)
get_all_orders: [] (Time taken: 0.29 seconds)
get_stoploss_orders: [] (Time taken: 0.15 seconds)
get_completed_orders: [] (Time taken: 0.19 seconds)
get_balance: 1000000.0 (Time taken: 0.14 seconds)
get_all_orders: [] (Time taken: 0.36 seconds)
get_stoploss_orders: [] (Time taken: 0.14 seconds)
get_pending_orders: [] (Time taken: 0.13 seconds)
get_portfolio: {} (Time taken: 0.12 seconds)
get_all_orders: [] (Time taken: 0.25 seconds)
get_stoploss_orders: [] (Time taken: 0.13 seconds)
get_completed_orders: [] (Time taken: 0.16 seconds)
get_balance: 1000000.0 (Time taken: 0.22 seconds)
get_stoploss_orders: [] (Time taken: 0.13 seconds)
get_stoploss_orders: [] (Time taken: 0.13 seconds)
get_pending_or

In [17]:
data_poller.stop_polling()

get_completed_orders: [] (Time taken: 0.14 seconds)


get_balance: 1000000.0 (Time taken: 0.13 seconds)
get_stoploss_orders: [] (Time taken: 0.21 seconds)
get_pending_orders: [] (Time taken: 0.14 seconds)
get_portfolio: {} (Time taken: 0.13 seconds)
get_stoploss_orders: [] (Time taken: 0.11 seconds)
get_completed_orders: [] (Time taken: 0.21 seconds)
get_balance: 1000000.0 (Time taken: 0.15 seconds)
get_stoploss_orders: [] (Time taken: 0.16 seconds)
